# Deep Mailing - Preparação de Dados

In [39]:
import pandas as pd
import dateutil.parser as parser
import os.path
import math
import logging
import numpy as np

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

arquivo_chamadas = "../data/mailing_completo.txt"
arquivo_df_pickled = "../intermediate/df.pickle"
arquivo_df_pickled_norm = "../intermediate/df.norm.pickle"
arquivo_df_pickled_norm_train = "../intermediate/df.norm.train.pickle"
arquivo_df_pickled_norm_test = "../intermediate/df.norm.test.pickle"

arquivo_df_pickled_norm_train_x = "../intermediate/df.norm.train.x.pickle"
arquivo_df_pickled_norm_train_y = "../intermediate/df.norm.train.y.pickle"
arquivo_df_pickled_norm_test_x = "../intermediate/df.norm.test.x.pickle"
arquivo_df_pickled_norm_test_y = "../intermediate/df.norm.test.y.pickle"

pd.options.display.max_columns = 50

Normalizado = False


In [40]:
def IsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def IsIntAndGreaterZero(s):
    return IsInt(s) and int(s)>0
    
def IsFloat(s):
    try: 
        float(s)
        return True
    except ValueError:
        return False
    
    
def IsDatetime(s):
    try: 
        parser.parse(s)
        return True
    except ValueError:
        return False
    
def ConverterInt(val):
    val = val.replace(",",".")
    if IsInt(val):
        return int(val)
    else:
        return 0

def ConverterFloat(val):
    val = val.replace(",",".")
    if IsFloat(val):
        return float(val)
    else:
        return 0
    
    
def ConverterData(val):
    if IsDatetime(val):
        return parser.parse(val)
    else:
        return val
    
def func_str(x):
    return str(x)

def func_strip(x):
    return str(x).strip()

def func_start_ALTA(x):
    return str(x).startswith('ALTA')

def IsCSVDataAvailable():
    return os.path.isfile(arquivo_df_pickled)

def IsNormDataAvailable():
    return os.path.isfile(arquivo_df_pickled_norm)

def IsTrainDataAvailable():
    return os.path.isfile(arquivo_df_pickled_norm_train)
    
def IsNumpyArrayDataAvailable():
    return os.path.isfile(arquivo_df_pickled_norm_train_x + ".npy")

def limpar_df(chamadas):
    del chamadas["CPF_CNPJ"]
    del chamadas["PRODUTO"]
    del chamadas["FILA"]
    del chamadas["STATUS_CONTRATO"]
    del chamadas["DETALHE_ORIGEM"]
    del chamadas["TELEFONE"]
    del chamadas["TELRUIM_RENITENCIA"]
    del chamadas["TELRUIM_DISCADOR"]
    del chamadas["OPERADORA"]
    del chamadas["ORIGEM_ULTIMA_ATUALIZACAO"]
    del chamadas["PRIMEIRA_ORIGEM"]
    del chamadas["ATRASO"] 
    del chamadas["VALOR"]
    del chamadas["DT_ENTRADA" ]
    del chamadas["NLOC"]
    del chamadas["SCORE_C"]
    del chamadas["SCORE_E"]
    del chamadas["RENDA"]
    del chamadas["DT_DEVOLUCAO"]
    del chamadas["VLRISCO"]
    del chamadas["SCORE_ZANC_C"]
    del chamadas["SCORE_ZANC_E"]
    del chamadas["SCORE_ZANC"]
    del chamadas["DATA_PRIMEIRA_ORIGEM"]
    del chamadas["DATA_ULTIMA_ATUALIZACAO"]
    del chamadas["ULT_ARQ_BUREAU"]
  
    return chamadas

def CreateColumn(cols, df, source_col, func):
    for col in cols:
        df['NORM_' + col] = df.apply(lambda row: 1 if func(row[source_col]) == col else 0, axis=1)
    return df    

def CreateLogColumn(df, source_col):
    df['NORM_' + source_col] = df.apply(lambda row: 1 if IsIntAndGreaterZero(row[source_col]) else 0, axis=1)
    return df

In [41]:
df_dtypes = {
    "CPF_CNPJ": "object",
    "CARTEIRA": "object",
    "SEGMENTO": "object",
    "PRODUTO": "object",
    "FILA": "object",
    "STATUS_CONTRATO": "object",
    "PROPENSAO": "object",
    "ORIGEM": "object",
    "DETALHE_ORIGEM": "object",
    "STATUS_BUREAU": "object",
    "STATUS_INTERNA": "object",
    "DDD": "object",
    "TELEFONE": "object",
    "TELRUIM_RENITENCIA": "object",
    "TELRUIM_DISCADOR": "object",
    "STATUS_TELEFONE": "object",
    "OPERADORA": "object",
    "ORIGEM_ULTIMA_ATUALIZACAO": "object",
    "PRIMEIRA_ORIGEM": "object"
}

converters = {
    "ATRASO":  ConverterInt,
    "VALOR": ConverterFloat,
    "DT_ENTRADA" : ConverterData,
    "NLOC": ConverterInt,
    "SCORE_C": ConverterInt,
    "SCORE_E": ConverterInt,
    "RENDA": ConverterFloat,
    "DT_DEVOLUCAO": ConverterData,
    "VLRISCO": ConverterFloat,
    "SCORE_ZANC_C": ConverterInt,
    "SCORE_ZANC_E": ConverterInt,
    "SCORE_ZANC": ConverterInt,
    "DATA_PRIMEIRA_ORIGEM": ConverterData,
    "DATA_ULTIMA_ATUALIZACAO": ConverterData,
    "TENTATIVAS": ConverterInt,
    "ULT_ARQ_BUREAU": ConverterData,
    "DATA_MAILING": ConverterData,
    "LIGACOES": ConverterInt,
    "CUP": ConverterInt
}

In [42]:
if not IsCSVDataAvailable():
    chamadas = pd.read_csv(arquivo_chamadas, sep="|", dtype=df_dtypes, converters = converters)
    chamadas = limpar_df(chamadas)
    chamadas.to_pickle(arquivo_df_pickled)
else:
    if not IsNormDataAvailable():    
        chamadas = pd.read_pickle(arquivo_df_pickled)
    else:
        chamadas = pd.read_pickle(arquivo_df_pickled_norm)
        Normalizado = True
        
logging.debug("Normalizado:{}".format(Normalizado))        

DEBUG:root:Normalizado:True


In [43]:
data_maxima_mailing = chamadas.DATA_MAILING.max()
data_minima_mailing = chamadas.DATA_MAILING.min()

print("Max:{} Min:{}".format(data_maxima_mailing, data_minima_mailing))

Max:2017-12-09 00:00:00 Min:2017-11-28 00:00:00


In [44]:
print("Normalizando Carteiras...")
if not Normalizado:
    Carteiras = set([x for x in chamadas.CARTEIRA.unique()[:-1] if len(x) == 3])
    chamadas = CreateColumn(Carteiras,chamadas, 'CARTEIRA', func_str)

Normalizando Carteiras...


In [45]:
print("Normalizando Segmentos...")
if not Normalizado:
    Segmentos = set([x.strip() for x in chamadas.SEGMENTO.unique()[:-1] if len(x.strip()) == 2])
    chamadas = CreateColumn(Segmentos,chamadas, 'SEGMENTO', func_strip)

Normalizando Segmentos...


In [46]:
print("Normalizando Chamadas...")
if not Normalizado:
    Propensao = set([x[:4] for x in chamadas.PROPENSAO.unique() if str(x).startswith("ALTA")])
    chamadas = CreateColumn(Propensao,chamadas, 'PROPENSAO', func_start_ALTA)

Normalizando Chamadas...


In [47]:
print("Normalizando Origem...")
if not Normalizado:
    Origem = set([x for x in chamadas.ORIGEM.unique()[:-1]])
    chamadas = CreateColumn(Origem,chamadas, 'ORIGEM', func_str)

Normalizando Origem...


In [48]:
print("Normalizando StatusBureau...")
if not Normalizado:
    StatusBureau = set([str(x) for x in chamadas.STATUS_BUREAU.unique()])
    chamadas = CreateColumn(StatusBureau,chamadas, 'STATUS_BUREAU', func_str)

Normalizando StatusBureau...


In [49]:
print("Normalizando StatusInterna...")
if not Normalizado:
    StatusInterna = set([str(x) for x in chamadas.STATUS_INTERNA.unique()[:-1]])
    chamadas = CreateColumn(StatusInterna,chamadas, 'STATUS_INTERNA', func_str)

Normalizando StatusInterna...


In [50]:
print("Normalizando Telefone...")
if not Normalizado:
    StatusTelefone = set([str(x) for x in chamadas.STATUS_TELEFONE.unique()[:-1]])
    chamadas = CreateColumn(StatusTelefone,chamadas, 'STATUS_TELEFONE', func_str)

Normalizando Telefone...


In [51]:
print("Normalizando DDD...")
if not Normalizado:
    DDD = set([str(x) for x in chamadas.DDD.unique()[:-1]])
    chamadas = CreateColumn(DDD,chamadas, 'DDD', func_str)

Normalizando DDD...


In [52]:
print("Normalizando LIGACOES E TAL...")
if not Normalizado:
    chamadas = CreateLogColumn(chamadas,'TENTATIVAS')
    chamadas = CreateLogColumn(chamadas,'LIGACOES')
    chamadas = CreateLogColumn(chamadas,'CUP')

Normalizando LIGACOES E TAL...


In [53]:
print("Removendo Campos desnecessarios e pickling...")
if not Normalizado:
    del chamadas['NUMERO']
    del chamadas['TENTATIVAS']
    del chamadas['LIGACOES']
    del chamadas['CUP']
    del chamadas['DDD']
    del chamadas['STATUS_TELEFONE']
    del chamadas['STATUS_INTERNA']
    del chamadas['STATUS_BUREAU']
    del chamadas['ORIGEM']
    del chamadas['SEGMENTO']
    del chamadas['PROPENSAO']
    del chamadas['CARTEIRA']
    chamadas.to_pickle(arquivo_df_pickled_norm)


Removendo Campos desnecessarios e pickling...


In [54]:
print("Criando Pickling de train e teste...")
if not IsTrainDataAvailable():
    chamadas_test = chamadas[(chamadas.DATA_MAILING >= data_maxima_mailing)]
    chamadas_test.to_pickle(arquivo_df_pickled_norm_test)
    del chamadas_test
    chamadas_train = chamadas[(chamadas.DATA_MAILING < data_maxima_mailing)]
    chamadas_train.to_pickle(arquivo_df_pickled_norm_train)
    del chamadas_train
   
del chamadas
chamadas_train = pd.read_pickle(arquivo_df_pickled_norm_train)
chamadas_test = pd.read_pickle(arquivo_df_pickled_norm_test)

Criando Pickling de train e teste...


In [55]:
print("Criando numpy arrays de train e teste...")
if not IsNumpyArrayDataAvailable():
    chamadas_train = chamadas_train[(chamadas_train.NORM_TENTATIVAS > 0)]
    header_chamadas_x = chamadas_train.loc[:, chamadas_train.columns.values[2]:'NORM_63'].head(1)
    with open(arquivo_df_pickled_norm_train_x+".txt","w") as f:
        counter = 0
        lista_header = list(header_chamadas_x.columns.values)
        for header in lista_header:
            f.write("{}-{}\n".format(counter,header))
            counter=counter+1
           
    chamadas_train_x = chamadas_train.loc[:, chamadas_train.columns.values[2]:'NORM_63'].as_matrix()
    chamadas_train_y = chamadas_train.NORM_CUP.as_matrix()
    del chamadas_train
    np.save(arquivo_df_pickled_norm_train_x,chamadas_train_x)
    np.save(arquivo_df_pickled_norm_train_y,chamadas_train_y)
    del chamadas_train_x
    del chamadas_train_y

    chamadas_test = chamadas_test[(chamadas_test.NORM_TENTATIVAS > 0)]
    chamadas_test_x = chamadas_test.loc[:, chamadas_test.columns.values[2]:'NORM_63'].as_matrix()
    chamadas_test_y = chamadas_test.NORM_CUP.as_matrix()
    del chamadas_test
    np.save(arquivo_df_pickled_norm_test_x,chamadas_test_x)
    np.save(arquivo_df_pickled_norm_test_y,chamadas_test_y)
    del chamadas_test_x
    del chamadas_test_y


Criando numpy arrays de train e teste...
